In [ ]:
from mylib.model import curtiss_model
import mylib.integration as integration

import numpy as np

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row

from ipywidgets import interact, FloatSlider, IntSlider

from mylib.model import curtiss_model
from mylib.integration import forward_euler, backward_euler

output_notebook(hide_banner=True)

# Stability, order and accuracy for non-stiff and stiff equations

## Curtiss and Hirschfelder

We consider the following problem :

\begin{equation} 
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(cos(t) - u(t)) \big) \quad{} \text{avec } k > 1\\ 
u(t_0)  & = u_0 
\end{aligned} 
\right. 
\end{equation}
and in the following, we will assume $t_0=0$.

## Stiffness

The exact solution is given by :

\begin{equation}
u(t) = \frac{k}{k^2+1} \bigg( k \cos(t) + \sin(t) \bigg) + c_0 \, e^{-k t} \quad{}
\text{avec} \quad{} c_0 = \bigg( u_0 -\frac{k}{k^2 + 1} \Big( k \cos(t_0) + \sin(t_0) \Big) \bigg)  e^{-k t_0} .  
\end{equation}


In [ ]:
def show_exact_sol():

    k = 50.
    uini = 2.
    tini = 0.
    tend = 10
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    uexa = cm.sol(uini, tini, texa)
 
    fig_sol = figure(x_range=(tini, tend), y_range=(-1.25, 2.25), plot_height=400, plot_width=900,
                     title="Exact solution of Curtiss and Hirschfelder equation")
    plt_sol = fig_sol.line(texa, uexa, color="Crimson", line_width=2)
    
    show(fig_sol, notebook_handle=True)
    
    def update(k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        uexa = cm.sol(uini, tini, texa)
        plt_sol.data_source.data = dict(x=texa, y=uexa)
        push_notebook()

    interact(update, k=FloatSlider(min=5, max=200, value=k, step=5))
    
show_exact_sol() 

## Explicit Euler

The explicit Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ can be written :

\begin{equation*}
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^n,U^n) \quad{} \text{where} \quad{} \Delta t = t^{n+1} - t^n
\end{aligned}
\right.
\end{equation*}

In [ ]:
def show_forward_euler_sol():

    k = 50.0
    uini = 2.
    #uini = k**2/(k**2+1)
    tini = 0.
    tend = 2.0
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 1000)
    uexa = cm.sol(uini, tini, texa)
 
    nt = 100
    sol = integration.forward_euler(tini, tend, nt, uini, fcn)
    
    uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-1.25, 2.25), plot_height=300, plot_width=900, title="Solution")
    plt_sol = fig_sol.line(texa, uexa, color="Crimson", line_width=2)
    plt_num = fig_sol.x(sol.t, sol.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol.t, sol.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Global error")
    plt_err = fig_err.x(sol.t, uerr, line_width=2, size=8)
    
    show(column(fig_sol,fig_err), notebook_handle=True)
    
    def update(nt, k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        #uini = k**2/(k**2+1)
        uexa = cm.sol(uini, tini, texa)
        sol = integration.forward_euler(tini, tend, nt, uini, fcn)
        uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
        plt_sol.data_source.data = dict(x=texa, y=uexa)
        plt_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_line_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_err.data_source.data = dict(x=sol.t, y=uerr)
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=0, max=100, value=k, step=5))
    
show_forward_euler_sol()

In [ ]:
def show_fe_order():

    k = 50.
    uini = 2
    tini = 0.
    tend = 2
    cm = curtiss_model(k)
    fcn = cm.fcn

    l_nt = [151, 1501, 15001, 150001]

    err = np.zeros((len(l_nt),2))
    err1 = np.zeros((len(l_nt),2))
    errinf = np.zeros((len(l_nt),2))

    for i in range(0, len(l_nt)):

        i_nt = l_nt[i]
        sol = integration.forward_euler(tini, tend, i_nt, uini, fcn)
        uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])

        norm_err = np.linalg.norm(uerr) / np.sqrt(l_nt[i])   
        norm_err1 = np.linalg.norm(uerr,1) / l_nt[i]
        norm_errinf = np.linalg.norm(uerr,np.inf) 

        err[i] = [(tend-tini)/(i_nt-1), norm_err] 
        err1[i] = [(tend-tini)/(i_nt-1), norm_err1] 
        errinf[i] = [(tend-tini)/(i_nt-1), norm_errinf] 

    ref = np.array([1.e-1, 1.e-2, 1.e-3, 1.e-3, 1.e-5, 1.e-7 ])

    f = figure(x_axis_type="log", y_axis_type="log", title="Error", x_range=[5.e-7, 5.e-2], plot_height=400)

    f.line(ref, ref, line_width=2, legend_label="slope 1", color="black", line_dash='dotted')
    f.x(err[:, 0], err[:, 1], size=10, line_width=2, legend_label="Norm l2")
    f.x(err1[:, 0], err1[:, 1], size=10, line_width=2, color="crimson", legend_label="Norm l1")
    f.x(errinf[:, 0], errinf[:, 1], size=10, line_width=2, color="green", legend_label="Norm linf")

    f.legend.location = "top_left"
    f.legend.click_policy="hide"

    show(f)
    
show_fe_order()

### Backward Euler

The backward Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ can be written :

\begin{equation*}
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^{n+1},U^{n+1}), \qquad \Delta t = t^{n+1} - t^n,
\end{aligned}
\right.
\end{equation*}

In [ ]:
def show_backward_euler_sol():

    k = 50.
    uini = 2.
    tini = 0.
    tend = 2
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    uexa = cm.sol(uini, tini, texa)
 
    nt = 100
    sol = integration.backward_euler(tini, tend, nt, uini, fcn)
    
    uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-1.25, 2.25), plot_height=300, plot_width=900, title="Solution")
    plt_sol = fig_sol.line(texa, uexa, color="Crimson", line_width=2)
    plt_num = fig_sol.x(sol.t, sol.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol.t, sol.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Global error")
    plt_err = fig_err.x(sol.t, uerr, line_width=2, size=8)
    
    show(column(fig_sol,fig_err), notebook_handle=True)
    
    def update(nt,k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        sol = integration.backward_euler(tini, tend, nt, uini, fcn)
        uexa = cm.sol(uini, tini, texa)
        uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
        plt_sol.data_source.data = dict(x=texa, y=uexa)
        plt_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_line_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_err.data_source.data = dict(x=sol.t, y=uerr)
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=0, max=100, value=k, step=5))
    
show_backward_euler_sol()

In [ ]:
def show_be_order():

    k = 50.
    uini = 2.
    tini = 0.
    tend = 2
    cm = curtiss_model(k)
    fcn = cm.fcn

    l_nt = [151, 1501, 15001, 150001]

    err = np.zeros((len(l_nt),2))
    err1 = np.zeros((len(l_nt),2))
    errinf = np.zeros((len(l_nt),2))

    for i in range(0, len(l_nt)):

        i_nt = l_nt[i]
        sol = integration.backward_euler(tini, tend, i_nt, uini, fcn)
        uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])

        norm_err = np.linalg.norm(uerr) / np.sqrt(l_nt[i])   
        norm_err1 = np.linalg.norm(uerr,1) / l_nt[i]
        norm_errinf = np.linalg.norm(uerr,np.inf) 

        err[i] = [(tend-tini)/(i_nt-1), norm_err] 
        err1[i] = [(tend-tini)/(i_nt-1), norm_err1] 
        errinf[i] = [(tend-tini)/(i_nt-1), norm_errinf] 

    ref = np.array([1.e-1, 1.e-2, 1.e-3, 1.e-3, 1.e-5, 1.e-7 ])

    f = figure(x_axis_type="log", y_axis_type="log", title="Error", x_range=[5.e-7, 5.e-2], plot_height=400)

    f.line(ref, ref, line_width=2, legend_label="slope 1", color="black", line_dash='dotted')
    f.x(err[:, 0], err[:, 1], size=10, line_width=2, legend_label="Norm l2")
    f.x(err1[:, 0], err1[:, 1], size=10, line_width=2, color="crimson", legend_label="Norm l1")
    f.x(errinf[:, 0], errinf[:, 1], size=10, line_width=2, color="green", legend_label="Norm linf")

    f.legend.location = "top_left"
    f.legend.click_policy="hide"

    show(f)
    
show_be_order()